In [75]:
from jwst import datamodels
import pysiaf
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u
import sys,os,re
import numpy as np
from astropy.modeling.models import Polynomial2D, Mapping, Shift
from pdastro import pdastroclass

fgs_apername = 'FGS1_FULL'
fgs_siaf = pysiaf.Siaf('FGS') 
fgs_aperture= fgs_siaf.apertures[fgs_apername]
fgs_x0=fgs_aperture.XSciRef-1.0
fgs_y0=fgs_aperture.YSciRef-1.0
print(f'{fgs_apername} V2Ref: {fgs_aperture.V2Ref}')
print(f'{fgs_apername} V3Ref: {fgs_aperture.V3Ref}')
print(f'{fgs_apername} V3IdlYAngle: {fgs_aperture.V3IdlYAngle}')

nrc_apername = 'NRCA1_FULL'
nrc_siaf = pysiaf.Siaf('NIRCAM') 
nrc_aperture= nrc_siaf.apertures[nrc_apername]
nrc_x0=nrc_aperture.XSciRef-1.0
nrc_y0=nrc_aperture.YSciRef-1.0
print(f'{nrc_apername} V2Ref: {nrc_aperture.V2Ref}')
print(f'{nrc_apername} V3Ref: {nrc_aperture.V3Ref}')
print(f'{nrc_apername} V3IdlYAngle: {nrc_aperture.V3IdlYAngle}')

FGS1_FULL V2Ref: 206.407
FGS1_FULL V3Ref: -697.765
FGS1_FULL V3IdlYAngle: -1.24120427
NRCA1_FULL V2Ref: 120.620865
NRCA1_FULL V3Ref: -527.541137
NRCA1_FULL V3IdlYAngle: -0.54644233


In [36]:
#fgsfilename = '/Volumes/Ext1/jhat/v2v3ref/hawki_v1/jw01476006004_06201_00002_guider1_jhat.fits'
#nrcfilename = '/Volumes/Ext1/jhat/v2v3ref/hawki_v1/jw01476006004_06101_00002_nrca1_jhat.fits'
fgsfilename = '/user/arest/v2v3example/jw01476006004_06201_00002_guider1_jhat.fits'
nrcfilename = '/user/arest/v2v3example/jw01476006004_06101_00002_nrca1_jhat.fits'

In [37]:
fgs_model = datamodels.ImageModel(fgsfilename)
fgs_detector_to_world=fgs_model.meta.wcs.get_transform('detector', 'world') 
fgs_detector_to_v2v3=fgs_model.meta.wcs.get_transform('detector', 'v2v3') 
fgs_world_to_v2v3=fgs_model.meta.wcs.get_transform('world','v2v3') 

nrc_model = datamodels.ImageModel(nrcfilename)
nrc_detector_to_world=nrc_model.meta.wcs.get_transform('detector', 'world') 


In [82]:
v2v3list = pdastroclass()
# calculate FGS V2/V3ref and V3IdlYAngle of image, and compare to nominal values
# caluclate FGS V2/V3 for center pixel +- dy
# save the V2/V3 coordinates in v2v3list
fgs_v2_0,fgs_v3_0 = fgs_detector_to_v2v3(fgs_x0,fgs_y0)
print(f'V2/V3ref siaf   {fgs_aperture.V2Ref:.4f} {fgs_aperture.V3Ref:.4f}')
print(f'V2/V3ref center {fgs_v2_0:.4f} {fgs_v3_0:.4f}')
dy = 0.02 # difference in pixel to go into +- y-direction
#y1 = fgs_y0+dy
#y2 = fgs_y0-dy
#x1=x2=fgs_x0
fgs_v2_1,fgs_v3_1 = fgs_detector_to_v2v3(fgs_x0,fgs_y0+dy)
fgs_v2_2,fgs_v3_2 = fgs_detector_to_v2v3(fgs_x0,fgs_y0-dy)

fgs_ix_0 = v2v3list.newrow({'name':'fgs_v2v3_0','v2':fgs_v2_0,'v3':fgs_v3_0})
fgs_ix_1 = v2v3list.newrow({'name':'fgs_v2v3_1','v2':fgs_v2_1,'v3':fgs_v3_1})
fgs_ix_2 = v2v3list.newrow({'name':'fgs_v2v3_2','v2':fgs_v2_2,'v3':fgs_v3_2})
fgs_V3IdlYAngle = np.degrees(np.arctan2(v2v3list.t.loc[fgs_ix_2,'v2']-v2v3list.t.loc[fgs_ix_1,'v2'], 
                                        v2v3list.t.loc[fgs_ix_2,'v3']-v2v3list.t.loc[fgs_ix_1,'v3']))
if fgs_V3IdlYAngle>90.0: fgs_V3IdlYAngle-=180
print(f'V3YIdlangle {fgs_V3IdlYAngle} (nominal: {fgs_aperture.V3IdlYAngle})')


v2v3list.write()

V2/V3ref siaf   206.4070 -697.7650
V2/V3ref center 206.4640 -697.9700
V3YIdlangle -1.2508171302552 (nominal: -1.24120427)
V3YIdlangle -1.2508171302552 (nominal: -1.24120427)
      name         v2          v3
fgs_v2v3_0 206.464000 -697.970000
fgs_v2v3_1 206.463969 -697.968602
fgs_v2v3_2 206.464031 -697.971398


0

In [83]:
# Convert x,y center of NIRCam into Ra,Dec, and then use FGS model to calculate the corresponding V2/V3 coordinates
# Do the same with center +- dy pixels

nrc_ra_0,nrc_dec_0 = nrc_detector_to_world(nrc_x0,nrc_y0)
nrc_ra_1,nrc_dec_1 = nrc_detector_to_world(nrc_x0,nrc_y0+dy)
nrc_ra_2,nrc_dec_2 = nrc_detector_to_world(nrc_x0,nrc_y0-dy)
nrc_v2_0,nrc_v3_0 = fgs_world_to_v2v3(nrc_ra_0,nrc_dec_0)
nrc_v2_1,nrc_v3_1 = fgs_world_to_v2v3(nrc_ra_1,nrc_dec_1)
nrc_v2_2,nrc_v3_2 = fgs_world_to_v2v3(nrc_ra_2,nrc_dec_2)
nrc_ix_0 = v2v3list.newrow({'name':'nrc_v2v3_0','v2':nrc_v2_0,'v3':nrc_v3_0})
nrc_ix_1 = v2v3list.newrow({'name':'nrc_v2v3_1','v2':nrc_v2_1,'v3':nrc_v3_1})
nrc_ix_2 = v2v3list.newrow({'name':'nrc_v2v3_2','v2':nrc_v2_2,'v3':nrc_v3_2})


In [89]:
# Rotate the V2/V3 so that the center FGS pixel and V3IdlYAngle agree with siaf
from astropy.modeling.rotations import Rotation2D
R = Rotation2D()
# calculate the differences in v2/v3 with respect to the center pixel v2/v3
v2diff=np.array(v2v3list.t['v2']-v2v3list.t.loc[fgs_ix_0,'v2'])
v3diff=np.array(v2v3list.t['v3']-v2v3list.t.loc[fgs_ix_0,'v3'])
# calculate the correction angle between the true fgs_V3IdlYAngle and the nominal one
angle = fgs_V3IdlYAngle-fgs_aperture.V3IdlYAngle
# Rotate the v2/v3 values, and add the *nominal* v2/v3 values
(v2diffrot,v3diffrot) = R.evaluate(v2diff,v3diff,angle*u.deg)
v2v3list.t['v2rot']=v2diffrot+fgs_aperture.V2Ref
v2v3list.t['v3rot']=v3diffrot+fgs_aperture.V3Ref
v2v3list.write()


fgs_V3IdlYAngle_rot = np.degrees(np.arctan2(v2v3list.t.loc[fgs_ix_2,'v2rot']-v2v3list.t.loc[fgs_ix_1,'v2rot'], 
                                            v2v3list.t.loc[fgs_ix_2,'v3rot']-v2v3list.t.loc[fgs_ix_1,'v3rot']))
if fgs_V3IdlYAngle_rot>90.0: fgs_V3IdlYAngle_rot-=180
print(f'FGS rotated V3YIdlangle {fgs_V3IdlYAngle_rot:.8f} from {fgs_V3IdlYAngle:.8f} (nominal: {fgs_aperture.V3IdlYAngle:.8f})')

print('###### Results:')
print(f'{nrc_apername} V2Ref: {v2v3list.t.loc[nrc_ix_0,"v2rot"]:.4f} (nominal: {nrc_aperture.V2Ref:.4f})')
print(f'{nrc_apername} V3Ref: {v2v3list.t.loc[nrc_ix_0,"v3rot"]:.4f} (nominal: {nrc_aperture.V3Ref:.4f})')
nrc_V3IdlYAngle = np.degrees(np.arctan2(v2v3list.t.loc[nrc_ix_2,'v2rot']-v2v3list.t.loc[nrc_ix_1,'v2rot'], 
                                        v2v3list.t.loc[nrc_ix_2,'v3rot']-v2v3list.t.loc[nrc_ix_1,'v3rot']))
if nrc_V3IdlYAngle>90.0: nrc_V3IdlYAngle-=180
print(f'{nrc_apername} V3YIdlangle {nrc_V3IdlYAngle:.8f} (nominal: {nrc_aperture.V3IdlYAngle:.8f})')


      name         v2          v3      v2rot       v3rot
fgs_v2v3_0 206.464000 -697.970000 206.407000 -697.765000
fgs_v2v3_1 206.463969 -697.968602 206.406970 -697.763602
fgs_v2v3_2 206.464031 -697.971398 206.407030 -697.766398
nrc_v2v3_0 120.680190 -527.857663 120.651732 -527.638273
nrc_v2v3_1 120.680183 -527.857037 120.651725 -527.637647
nrc_v2v3_2 120.680196 -527.858290 120.651738 -527.638900
FGS rotated V3YIdlangle -1.24120427 from -1.25081713 (nominal: -1.24120427)
###### Results:
NRCA1_FULL V2Ref: 120.6517 (nominal: 120.6209)
NRCA1_FULL V3Ref: -527.6383 (nominal: -527.5411)
NRCA1_FULL V3YIdlangle -0.55385852 (nominal: -0.54644233)
